In [1]:
import urllib.request
import json
import datetime
import csv
import time
import codecs

In [2]:
app_id = "1192796350809849"
app_secret = "585a729a44bb655f2cd6ef954e191a79"
access_token = app_id +"|"+ app_secret
group_id = '172177269650835'#'publicserviceinnovationblog'

In [3]:
def request_until_completed(url):
    req = urllib.request.Request(url)
    success = False
    while success is False:
        try: 
            response = urllib.request.urlopen(req)
            if response.getcode() == 200:
                success = True
        except Exception as e:
            print(e)
            time.sleep(5)

            print("Error for URL %s: %s" % (url, datetime.datetime.now()))
            print("Retrying.")

    return response.read().decode(response.headers.get_content_charset())

In [4]:
def unicode_normalize(text):
    return text.translate({ 0x2018:0x27, 0x2019:0x27, 0x201C:0x22, 0x201D:0x22,
                            0xa0:0x20 })


In [5]:
def get_fb_group_data(group_id, access_token, num_statuses):

    # Construct the URL string; see http://stackoverflow.com/a/37239851 for
    # Reactions parameters
    base = "https://graph.facebook.com"     # main url
    node = "/" + group_id + "/feed" 
    parameters = "/?fields=message,link,created_time,type,name,id,likes.limit(1).summary(true),comments.limit(1).summary(true),shares&limit=%s&access_token=%s" % (num_statuses, access_token) # changed
    url = base + node + parameters
    print(url)

    # retrieve data
    data = json.loads(request_until_completed(url))

    return data
print(get_fb_group_data(group_id, access_token, 1))

https://graph.facebook.com/172177269650835/feed/?fields=message,link,created_time,type,name,id,likes.limit(1).summary(true),comments.limit(1).summary(true),shares&limit=1&access_token=1192796350809849|585a729a44bb655f2cd6ef954e191a79
{'data': [{'name': 'একটি পরিচ্ছন্ন সরকারি হাসপাতাল', 'id': '172177269650835_635087893359768', 'type': 'link', 'comments': {'data': [{'id': '635156986686192', 'from': {'name': 'Manik Mahmud', 'id': '10153892048956582'}, 'created_time': '2016-12-18T09:05:56+0000', 'message': 'Sirazul Khan স্যার অনুগ্রহ করে মন্তব্য করতে পারেন।'}], 'summary': {'total_count': 4, 'can_comment': False, 'order': 'chronological'}, 'paging': {'cursors': {'after': 'WTI5dGJXVnVkRjlqZAFhKemIzSTZAOak0xTVRVMk9UZAzJOamcyTVRreU9qRTBPREl3TlRFNU5UWT0ZD', 'before': 'WTI5dGJXVnVkRjlqZAFhKemIzSTZAOak0xTVRVMk9UZAzJOamcyTVRreU9qRTBPREl3TlRFNU5UWT0ZD'}, 'next': 'https://graph.facebook.com/v2.8/172177269650835_635087893359768/comments?access_token=1192796350809849%7C585a729a44bb655f2cd6ef954e191a79

In [6]:
def get_reactions_of_each_status(status_id, access_token):

    # See http://stackoverflow.com/a/37239851 for Reactions parameters
        # Reactions are only accessable at a single-post endpoint

    base = "https://graph.facebook.com/v2.6"
    node = "/%s" % status_id
    reactions = "/?fields=" \
            "reactions.type(LIKE).limit(0).summary(total_count).as(like)" \
            ",reactions.type(LOVE).limit(0).summary(total_count).as(love)" \
            ",reactions.type(WOW).limit(0).summary(total_count).as(wow)" \
            ",reactions.type(HAHA).limit(0).summary(total_count).as(haha)" \
            ",reactions.type(SAD).limit(0).summary(total_count).as(sad)" \
            ",reactions.type(ANGRY).limit(0).summary(total_count).as(angry)"
    parameters = "&access_token=%s" % access_token
    url = base + node + reactions + parameters

    # retrieve data
    data = json.loads(request_until_completed(url))

    return data


In [7]:


def process_fb_status(status, access_token):

    # The status is now a Python dictionary, so for top-level items,
    # we can simply call the key.

    # Additionally, some items may not always exist,
    # so must check for existence first

    status_id = status['id']
    status_message = '' if 'message' not in status.keys() else \
            unicode_normalize(status['message'])
    link_name = '' if 'name' not in status.keys() else \
            unicode_normalize(status['name'])
    status_type = status['type']
    status_link = '' if 'link' not in status.keys() else \
            unicode_normalize(status['link'])

    # Time needs special care since a) it's in UTC and
    # b) it's not easy to use in statistical programs.

    status_published = datetime.datetime.strptime(
            status['created_time'],'%Y-%m-%dT%H:%M:%S+0000')
    status_published = status_published + \
            datetime.timedelta(hours=-5) # EST
    status_published = status_published.strftime(
            '%Y-%m-%d %H:%M:%S') # best time format for spreadsheet programs

    # Nested items require chaining dictionary keys.

    num_reactions = 0 if 'reactions' not in status else \
            status['reactions']['summary']['total_count']
    num_comments = 0 if 'comments' not in status else \
            status['comments']['summary']['total_count']
    num_shares = 0 if 'shares' not in status else status['shares']['count']

    # Counts of each reaction separately; good for sentiment
    # Only check for reactions if past date of implementation:
    # http://newsroom.fb.com/news/2016/02/reactions-now-available-globally/

    reactions = get_reactions_of_each_status(status_id, access_token) if \
            status_published > '2016-02-24 00:00:00' else {}

    num_likes = 0 if 'like' not in reactions else \
            reactions['like']['summary']['total_count']

    # Special case: Set number of Likes to Number of reactions for pre-reaction
    # statuses

    num_likes = num_reactions if status_published < '2016-02-24 00:00:00' \
            else num_likes

    def get_num_total_reactions(reaction_type, reactions):
        if reaction_type not in reactions:
            return 0
        else:
            return reactions[reaction_type]['summary']['total_count']

    num_loves = get_num_total_reactions('love', reactions)
    num_wows = get_num_total_reactions('wow', reactions)
    num_hahas = get_num_total_reactions('haha', reactions)
    num_sads = get_num_total_reactions('sad', reactions)
    num_angrys = get_num_total_reactions('angry', reactions)

    # Return a tuple of all processed data

    return (status_id, status_message, link_name, status_type, status_link,
            status_published, num_reactions, num_comments, num_shares,
            num_likes, num_loves, num_wows, num_hahas, num_sads, num_angrys)


In [11]:
def scrape_fb_group_status(group_id, access_token):
    with open('%s_facebook_statuses.csv' % group_id, 'w', newline='',encoding='utf-8') as file:
        w = csv.writer(file)
        w.writerow(["status_id", "status_message", "link_name", "status_type",
                    "status_link", "status_published", "num_reactions", 
                    "num_comments", "num_shares", "num_likes", "num_loves", 
                    "num_wows", "num_hahas", "num_sads", "num_angrys"])

        has_next_page = True
        num_processed = 0   # keep a count on how many we've processed
        scrape_starttime = datetime.datetime.now()

        print("Scraping %s Facebook Page: %s\n" % (group_id, scrape_starttime))

        statuses = get_fb_group_data(group_id, access_token, 100)

        while has_next_page:
            for status in statuses['data']:

                # Ensure it is a status with the expected metadata
#                 if 'reactions' in status:
                w.writerow(process_fb_status(status,
                        access_token))

                # output progress occasionally to make sure code is not
                # stalling
                num_processed += 1
                if num_processed % 100 == 0:
                    print("%s Statuses Processed: %s" % \
                        (num_processed, datetime.datetime.now()))

            # if there is no next page, we're done.
            if 'paging' in statuses.keys():
                statuses = json.loads(request_until_completed(
                                        statuses['paging']['next']))
            else:
                has_next_page = False


        print("\nDone!\n%s Statuses Processed in %s" % \
            (num_processed, datetime.datetime.now() - scrape_starttime))



In [12]:
    scrape_fb_group_status(group_id, access_token)

Scraping 172177269650835 Facebook Page: 2016-12-19 08:33:52.869184

https://graph.facebook.com/172177269650835/feed/?fields=message,link,created_time,type,name,id,likes.limit(1).summary(true),comments.limit(1).summary(true),shares&limit=100&access_token=1192796350809849|585a729a44bb655f2cd6ef954e191a79
100 Statuses Processed: 2016-12-19 08:35:06.527065
200 Statuses Processed: 2016-12-19 08:36:12.452829
300 Statuses Processed: 2016-12-19 08:37:17.915326
400 Statuses Processed: 2016-12-19 08:38:22.971181
500 Statuses Processed: 2016-12-19 08:39:29.060562
600 Statuses Processed: 2016-12-19 08:40:48.207082
700 Statuses Processed: 2016-12-19 08:42:08.045097
800 Statuses Processed: 2016-12-19 08:43:19.311511
900 Statuses Processed: 2016-12-19 08:44:25.086511
1000 Statuses Processed: 2016-12-19 08:45:38.825358
1100 Statuses Processed: 2016-12-19 08:46:43.686098
1200 Statuses Processed: 2016-12-19 08:48:01.005216
1300 Statuses Processed: 2016-12-19 08:49:13.418057
1400 Statuses Processed: 2016